
   <h1><center>Twitter topic classification Hackathon</center></h1>
   <h2><center>Includes Topic Modelling</center><h2>
  

In [81]:
## Importing all the standard libraries , 
## Pandas, numpy arrays, Regular Expression, NLTK for text processing, Sklearn for Classifiction models

import pandas as pd
import numpy as np
import sklearn
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score,f1_score


<h1><center>###  Importing the CSV file from the Local PC ###</center></h1>
<end>Data set has been read in to the DataFrame variable twd(Twitter Data)</end>

<end> Please paste the link of your file in the same function</end>


In [2]:
twd=pd.read_csv(r'C:/Users/pc/Desktop/Hackathon/twcs/twcs.csv')

##### Displaying first five observations of the table. 

In [3]:
twd.head()

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
0,1,sprintcare,False,Tue Oct 31 22:10:47 +0000 2017,@115712 I understand. I would like to assist y...,2,3.0
1,2,115712,True,Tue Oct 31 22:11:45 +0000 2017,@sprintcare and how do you propose we do that,NaN,1.0
2,3,115712,True,Tue Oct 31 22:08:27 +0000 2017,@sprintcare I have sent several private messag...,1,4.0
3,4,sprintcare,False,Tue Oct 31 21:54:49 +0000 2017,@115712 Please send us a Private Message so th...,3,5.0
4,5,115712,True,Tue Oct 31 21:49:35 +0000 2017,@sprintcare I did.,4,6.0


### Here in this Data set we need the text data and either it is a good or bad statement by customer
  - ##### Hence discarding the rest of the columns

In [4]:
twd.columns

Index(['tweet_id', 'author_id', 'inbound', 'created_at', 'text',
       'response_tweet_id', 'in_response_to_tweet_id'],
      dtype='object')

##### Dropping the unecessary columns 

In [5]:
twd.drop(['tweet_id', 'author_id', 'created_at','response_tweet_id', 'in_response_to_tweet_id'], axis=1, inplace=True)

#####   Dataset after removing the columns

In [6]:
twd.head()

,inbound,text
0,False,@115712 I understand. I would like to assist y...
1,True,@sprintcare and how do you propose we do that
2,True,@sprintcare I have sent several private messag...
3,False,@115712 Please send us a Private Message so th...
4,True,@sprintcare I did.


##### Checking the total number of observations and columns present in the data set.

In [7]:
twd.shape

(2811774, 2)

###### Only Ten thousand observations has been taken to develope the model. 
######  The New data frame variable name  is "twd_sample "(twitter data Sample) consisting 10,000 observations.
######  It restricted because of memory limitations , if it is not so, u can pass the whole data set like """twd.iloc[:,:].copy()"""


In [8]:
twd_sample=twd.iloc[2070000:2080000,:].copy()

In [9]:
twd_sample.head()

,inbound,text
2070000,True,"@idea_cares Sneha, when i call some i hear you..."
2070001,False,@336024 We apologies for the trouble. We would...
2070002,True,"@idea_cares u irritating idiots, y the fuck pr..."
2070003,False,@307570 Sorry we gave you a chance of being up...
2070004,True,@idea_cares He told me - Sir don't repeat your...


##### Checking whether our dataset consists of NaNs 

In [10]:
twd_sample.isna().sum()

inbound    0
text       0
dtype: int64

##### Counting the number of Postive and Negative comments avialable in the DataFrame  

In [11]:
twd_sample['inbound'].value_counts()

True     5428
False    4572
Name: inbound, dtype: int64

##### Writing the generalised function to remove the uncessary characters, stopwords

In [12]:
def remove_noise(input_text):
    
    lema=WordNetLemmatizer()                                ### World Lematizer 

    input_text = re.sub('<.*?>', ' ', (input_text))         ### Removing html and xml tags in text data and replacing with blanks
    input_text = str(input_text).replace("<br />", " ")     ###  Removing Line Breakage tags
    input_text = str(input_text).replace(",", "")           ### Removing punctuations and replacing with blanks
    input_text = str(input_text).replace("\'" ,"")
    input_text = str(input_text).replace("\\", "")
    input_text = str(input_text).replace("-", "")
    input_text = str(input_text).replace(".", "")
    input_text  =re.sub(r'\s+', ' ',input_text )            ### Removing multiple spaces
     
    
    input_text = re.sub("[^a-zA-Z]"," ",input_text)         ## Considering only alphabets rest is replaced by spaces
    input_text = input_text.lower()                         ## Lowering the text
    words      = input_text.split()                         ## Splitting the text
    noise_free_words =  [w for w in words if w not in set(stopwords.words('english'))] ## Removing Stop words
    lower_words=[lema.lemmatize(w) for w in noise_free_words]                          ## Apllying Lemmatization to the words                        
    lower_words=' '.join(lower_words)
                                   
    return lower_words   

##### Splitting the Data into 70% Train, 30% Test by passing text data in to  
##### Independent variable "X" and True or False in to Targer variable "y"

In [13]:
from sklearn.model_selection import train_test_split

X=twd_sample['text']
y=twd_sample['inbound']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

##### Removing the noise in the Train dataset

In [14]:
X_train=X_train.apply(remove_noise)

In [15]:
X_train.head()

2079069    centurylinkhelp calling calling simple account...
2072603    sorry hear please dm u information experience ...
2077738    apology kazuma happy look please follow dm u a...
2071579                           wed like look device using
2075058    day ported number vodafone call failing cancel...
Name: text, dtype: object

##### Removing the noise in the Test dataset

In [16]:
X_test=X_test.apply(remove_noise)

In [17]:
X_test.head()

2076252              glad could help gillian good day steven
2074684    british airway would nice id settle reply dm f...
2071731    birthday lunch help one fave sandwich mcrib po...
2074742                             british airway ba lhrdxb
2074521                             weve received dm respond
Name: text, dtype: object

##### Converting of Words in the observations to matrix representation using Tfidf Vectorizer
- Word Frequencies with TfidfVectorizer
- This stands for “Term Frequency – Inverse Document Frequency" which are the components of the resulting -  scores assigned to each word.
- Inverse Document Frequency: This downscales words that appear a lot across documents.

In [22]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC

X_tr=X_train
X_ts=X_test
y_tr=y_train
y_ts=y_test

tf=TfidfVectorizer(max_df=0.85, min_df=3,max_features=1600, stop_words='english')

X_tr=tf.fit_transform(X_tr).toarray()

X_ts=tf.fit_transform(X_ts).toarray()

  ##### Note if X_train and X_test column "(m,n)"  "(k,n)" dimension is not matching, 
  - please change the max_features in Tfidf Vectorizer paramter in the above
  - Remember please give high features  to learn more from the dataset 

In [53]:
if X_tr.shape[1]==X_ts.shape[1]:
    print('The shape of X_train is :',X_tr.shape,'\n','The shape of X_test is :',X_ts.shape)
else:
    print('Please change the max_features hyper paramter in the Tfidf vectorizer and run the code again')

The shape of X_train is : (7000, 1600) 
 The shape of X_test is : (3000, 1600)


## Logistic Regression
- Machine Learning supervised algorithm to classify the positive and negative statemnts
- In this the default threshold taken as 0.5

In [38]:
from sklearn.linear_model import LogisticRegression
Logr=LogisticRegression(solver ='liblinear',class_weight='balanced')
Logr.fit(X_tr,y_tr)


LogisticRegression(class_weight='balanced', solver='liblinear')

### Test Results report  Logistic includes Accuracy, F1 Score, Confusion matrix
- Accuracy= TP+TN/(TP+TN+FP+FN)
- Precision= TP/(TP+FP)
- Recall= TP/(TP+FN)
- F1 Score= 2*(PrecisionxRecall)/(Precision+Recall)

In [65]:
y_pred=Logr.predict(X_ts)

print(classification_report(y_pred,y_ts),end=" \n")

print('Confusion Matrix :','\n',confusion_matrix(y_pred,y_ts),'\n')

print('The F1_score of the model on test _data_set:',f1_score(y_pred,y_ts),'\n')

print('The accuracy_score of the model on test _data_set:',accuracy_score(y_pred,y_ts))

              precision    recall  f1-score   support

       False       0.42      0.58      0.49      1010
        True       0.74      0.60      0.66      1990

    accuracy                           0.59      3000
   macro avg       0.58      0.59      0.58      3000
weighted avg       0.63      0.59      0.60      3000
 
Confusion Matrix : 
 [[ 585  425]
 [ 792 1198]] 

The F1_score of the model on test _data_set: 0.6631608081926378 

The accuracy_score of the model on test _data_set: 0.5943333333333334


## Decision Tree classifier
- _Intially performing Randomized Search on the pramaters available in the DecisionTree Model_
- _Also in Randomzed Search N-iterations has been taken only 5 due to memory constrains_
-  _u can good to go with higher iterations_
- _After getting best paramters those have to be substituted in to the algorithm_

In [31]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier

params = {'max_leaf_nodes': list(range(2, 100)), 'min_samples_split': [2, 3, 4, 5],'max_depth':list(range(8, 100,2))}
dt = DecisionTreeClassifier()
clf=RandomizedSearchCV(estimator = dt, param_distributions = params, n_iter = 8, cv = 5,\
                               verbose=3, random_state=42, n_jobs = -1)

clf.fit(X_tr, y_tr)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   30.2s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:  1.4min finished


RandomizedSearchCV(cv=5, estimator=DecisionTreeClassifier(), n_iter=8,
                   n_jobs=-1,
                   param_distributions={'max_depth': [8, 10, 12, 14, 16, 18, 20,
                                                      22, 24, 26, 28, 30, 32,
                                                      34, 36, 38, 40, 42, 44,
                                                      46, 48, 50, 52, 54, 56,
                                                      58, 60, 62, 64, 66, ...],
                                        'max_leaf_nodes': [2, 3, 4, 5, 6, 7, 8,
                                                           9, 10, 11, 12, 13,
                                                           14, 15, 16, 17, 18,
                                                           19, 20, 21, 22, 23,
                                                           24, 25, 26, 27, 28,
                                                           29, 30, 31, ...],
                                

#### Use the below  best classifer fit for the decision tree

In [40]:
clf.best_estimator_


DecisionTreeClassifier(max_depth=92, max_leaf_nodes=98, min_samples_split=4)

In [67]:
from sklearn.tree import DecisionTreeClassifier

Dt=DecisionTreeClassifier(max_depth=92, max_leaf_nodes=98,min_samples_split=4 )
Dt.fit(X_tr,y_tr)


DecisionTreeClassifier(max_depth=92, max_leaf_nodes=98, min_samples_split=4)

### Test Results report  Decision Tree includes Accuracy, F1 Score, Confusion matrix

In [68]:
y_pred=Dt.predict(X_ts)

print(classification_report(y_pred,y_ts),end=" \n")

print('Confusion Matrix :','\n',confusion_matrix(y_pred,y_ts),'\n')

print('The F1_score of the model on test _data_set:',f1_score(y_pred,y_ts),'\n')

print('The accuracy_score of the model on test _data_set:',accuracy_score(y_pred,y_ts))

              precision    recall  f1-score   support

       False       0.38      0.59      0.46       880
        True       0.78      0.60      0.67      2120

    accuracy                           0.59      3000
   macro avg       0.58      0.59      0.57      3000
weighted avg       0.66      0.59      0.61      3000
 
Confusion Matrix : 
 [[ 519  361]
 [ 858 1262]] 

The F1_score of the model on test _data_set: 0.6743254074271974 

The accuracy_score of the model on test _data_set: 0.5936666666666667


##### Cross validation on Train Dataset
- _Checking the cross validation on the Train dataset_.
- _The model is performing well  on Train data set because it is trained on 70% of observations_.

In [60]:
from sklearn.model_selection import cross_val_score 
Dt=DecisionTreeClassifier()
accuracies = cross_val_score(estimator = Dt, X = X_tr,\
     y = y_tr, cv = 5,verbose=2)
print("Accuracy Mean {} Accuracy Variance \
     {}".format(accuracies.mean(),accuracies.std()))


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  ................................................................
[CV] ................................................. , total=  16.4s
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   16.3s remaining:    0.0s


[CV] ................................................. , total=  19.6s
[CV]  ................................................................
[CV] ................................................. , total=  14.2s
[CV]  ................................................................
[CV] ................................................. , total=  15.6s
[CV]  ................................................................
[CV] ................................................. , total=  16.2s
Accuracy Mean 0.8344285714285714 Accuracy Variance      0.0024074713637646616


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:  1.4min finished


## Random Forest classifier
- _Intially performing Randomized Search on the pramaters available in the Random_forest Model.
- _Also in Randomzed Search N_iterations has been taken only 5 due to memory constrains_. 
-  _You can good to go with higher iterations_.
- _After getting best paramters those have to be substituted in to the algorithm_.

In [52]:
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 1000, num = 100)]
# Number of features to consider at every split
max_features = [int(x) for x in np.linspace(start = 20, stop = 100, num = 10)]
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(2, 100, num = 1)]

# Minimum number of samples required to split a node
min_samples_split = [2, 3,4,5,6, 7,8,9,10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [ 2, 3,4,5,6,7,8,9]
# Method of selecting samples for training each tree
bootstrap = [True, False]


# Create the random grid
params = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}


# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestClassifier()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = params, n_iter = 5, cv = 5,\
                               verbose=3, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_tr, y_tr)

Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  19 out of  25 | elapsed:  2.1min remaining:   39.9s
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:  2.3min finished


RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(), n_iter=5,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [2],
                                        'max_features': [20, 28, 37, 46, 55, 64,
                                                         73, 82, 91, 100],
                                        'min_samples_leaf': [2, 3, 4, 5, 6, 7,
                                                             8, 9],
                                        'min_samples_split': [2, 3, 4, 5, 6, 7,
                                                              8, 9, 10],
                                        'n_estimators': [100, 109, 118, 127,
                                                         136, 145, 154, 163,
                                                         172, 181, 190, 200,
                                                         209, 218, 227, 236,
         

In [54]:
rf_random.best_estimator_

RandomForestClassifier(bootstrap=False, max_depth=2, max_features=91,
                       min_samples_leaf=4, min_samples_split=7,
                       n_estimators=390)

In [55]:
from sklearn.ensemble import RandomForestClassifier

RF=RandomForestClassifier(bootstrap=False, max_depth=2, max_features=91,
                       min_samples_leaf=4, min_samples_split=7,
                       n_estimators=390)
RF.fit(X_tr,y_tr)


RandomForestClassifier(bootstrap=False, max_depth=2, max_features=91,
                       min_samples_leaf=4, min_samples_split=7,
                       n_estimators=390)

### Test Results report  Random Forest includes Accuracy, F1 Score, Confusion matrix

In [57]:
y_pred=RF.predict(X_ts)

from sklearn.metrics import confusion_matrix,classification_report,accuracy_score

print(classification_report(y_pred,y_ts),end=" \n")

print('Confusion Matrix :','\n',confusion_matrix(y_pred,y_ts),'\n')

print('The F1_score of the model on test _data_set:',f1_score(y_pred,y_ts),'\n')

print('The accuracy_score of the model on test _data_set:',accuracy_score(y_pred,y_ts))

              precision    recall  f1-score   support

       False       0.01      0.83      0.03        23
        True       1.00      0.54      0.70      2977

    accuracy                           0.55      3000
   macro avg       0.51      0.68      0.37      3000
weighted avg       0.99      0.55      0.70      3000
 
Confusion Matrix : 
 [[  19    4]
 [1358 1619]] 

The F1_score of the model on test _data_set: 0.7039130434782609 

The accuracy_score of the model on test _data_set: 0.546


In [117]:
from sklearn.model_selection import cross_val_score 
accuracies = cross_val_score(estimator = RF, X = X_tr,\
     y = y_tr, cv = 5)
print("Accuracy Mean {} Accuracy Variance \
     {}".format(accuracies.mean(),accuracies.std()))


Accuracy Mean 0.91425 Accuracy Variance      0.007988272654335235


## Support Vector Machines

In [64]:
from sklearn.svm import SVC

svc = SVC(kernel='rbf',C=1,gamma=10)
svc.fit(X_tr,y_tr)


SVC(C=1, gamma=10)

## Test Results Support Vector Machines Accuracy,F1_Score Confusion Matrix

In [69]:
y_pred=svc.predict(X_ts)


print(classification_report(y_pred,y_ts),end=" \n")

print('Confusion Matrix :','\n',confusion_matrix(y_pred,y_ts),'\n')

print('The F1_score of the model on test _data_set:',f1_score(y_pred,y_ts),'\n')

print('The accuracy_score of the model on test _data_set:',accuracy_score(y_pred,y_ts))

              precision    recall  f1-score   support

       False       0.00      0.00      0.00         1
        True       1.00      0.54      0.70      2999

    accuracy                           0.54      3000
   macro avg       0.50      0.27      0.35      3000
weighted avg       1.00      0.54      0.70      3000
 
Confusion Matrix : 
 [[   0    1]
 [1377 1622]] 

The F1_score of the model on test _data_set: 0.7018606663781913 

The accuracy_score of the model on test _data_set: 0.5406666666666666


## Naive bayes MultinomialNB

In [74]:
from sklearn.naive_bayes import MultinomialNB,GaussianNB

navby=MultinomialNB()
navby.fit(X_tr,y_tr)


MultinomialNB()

## Test Results Support Vector Machines Accuracy,F1_Score< Confusion Matrix

In [75]:
y_pred=navby.predict(X_ts)

print(classification_report(y_pred,y_ts),end=" \n")

print('Confusion Matrix :','\n',confusion_matrix(y_pred,y_ts),'\n')

print('The F1_score of the model on test _data_set:',f1_score(y_pred,y_ts),'\n')

print('The accuracy_score of the model on test _data_set:',accuracy_score(y_pred,y_ts))

              precision    recall  f1-score   support

       False       0.38      0.55      0.45       950
        True       0.74      0.59      0.65      2050

    accuracy                           0.58      3000
   macro avg       0.56      0.57      0.55      3000
weighted avg       0.63      0.58      0.59      3000
 
Confusion Matrix : 
 [[ 527  423]
 [ 850 1200]] 

The F1_score of the model on test _data_set: 0.6534168254832561 

The accuracy_score of the model on test _data_set: 0.5756666666666667


## Cross validation for Naive bayes

In [63]:
from sklearn.model_selection import cross_val_score 
accuracies = cross_val_score(estimator = navby, X = X_tr,\
     y = y_tr, cv = 7,verbose=2)
print("Accuracy Mean {} Accuracy Variance \
     {}".format(accuracies.mean(),accuracies.std()))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


[CV]  ................................................................
[CV] ................................................. , total=   0.1s
[CV]  ................................................................
[CV] ................................................. , total=   0.1s
[CV]  ................................................................
[CV] ................................................. , total=   0.1s
[CV]  ................................................................
[CV] ................................................. , total=   0.1s
[CV]  ................................................................
[CV] ................................................. , total=   0.1s
[CV]  ................................................................
[CV] ................................................. , total=   0.1s
[CV]  ................................................................
[CV] ................................................. , total=   0.1s
Accura

[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:    0.8s finished


## XG Boost classifier
- _Intially performing Randomized Search on the pramaters available in the XG Boost Model.
- _Also in Randomzed Search N_iterations has been taken only 5 due to memory constrains_. 
-  _You can good to go with higher iterations_.
- _After getting best paramters those have to be substituted in to the algorithm_.

In [51]:
## Hyper Parameter Optimization
import xgboost as xgb
params={
 "learning_rate"    : [0.01,0.05, 0.10, 0.15, 0.20, 0.25, 0.30 ] ,
 "max_depth"        : [ 3, 4, 5, 6, 8, 10, 12, 15],
 "min_child_weight" : [ 1, 3, 5, 7 ],
 "gamma"            : [ 0.0, 0.1, 0.2 , 0.3, 0.4 ],
 "colsample_bytree" : [ 0.3, 0.4, 0.5 , 0.7 ],
    "reg_alpha"     : [1,3,5,7,10],
     "reg_lambda"   : [1,3,5,7,10]
}
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
classifier=xgb.XGBClassifier()
random_search=RandomizedSearchCV(classifier,param_distributions=params,n_iter=5,scoring='f1_macro',n_jobs=-1,cv=5,verbose=3)
random_search.fit(X_tr,y_tr)

Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  19 out of  25 | elapsed:  5.3min remaining:  1.7min
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:  6.4min finished


RandomizedSearchCV(cv=5,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, gamma=None,
                                           gpu_id=None, importance_type='gain',
                                           interaction_constraints=None,
                                           learning_rate=None,
                                           max_delta_step=None, max_depth=None,
                                           min_child_weight=None, missing=nan,
                                           monotone_constraints=None,
                                           n_estimators=100,...
                                           subsample=None, tree_method=None,
                                           validate_parameters=None,
                                   

- Best Estimator after hyper paramter tuning

In [57]:
random_search.best_estimator_

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.4, gamma=0.2, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.3, max_delta_step=0, max_depth=12,
              min_child_weight=3, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1, random_state=0,
              reg_alpha=7, reg_lambda=10, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [50]:
import xgboost as xgb

clf = xgb.XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.4, eval_metric='error',
              gamma=0.0, gpu_id=-1, importance_type='gain',
              interaction_constraints='', learning_rate=0.2, max_delta_step=0,
              max_depth=7, min_child_weight=5,
             num_parallel_tree=1, random_state=0, reg_alpha=3, reg_lambda=1,
              scale_pos_weight=1, subsample=1, tree_method='exact',
              validate_parameters=1, verbosity=None)


clf.fit(X_tr,y_tr)


C:\Users\pc\Anaconda3\lib\site-packages\dask\config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
C:\Users\pc\Anaconda3\lib\site-packages\distributed\config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.4, eval_metric='error',
              gamma=0.0, gpu_id=-1, importance_type='gain',
              interaction_constraints='', learning_rate=0.2, max_delta_step=0,
              max_depth=7, min_child_weight=5, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=0,
              num_parallel_tree=1, random_state=0, reg_alpha=3, reg_lambda=1,
              scale_pos_weight=1, subsample=1, tree_method='exact',
              validate_parameters=1, verbosity=None)

## Test Results XG Boost Accuracy,F1_Score< Confusion Matrix

In [79]:
y_pred=clf.predict(X_ts)


print(classification_report(y_pred,y_ts),end=" \n")

print('Confusion Matrix :','\n',confusion_matrix(y_pred,y_ts),'\n')


print('The f1_score of the model:',f1_score(y_pred,y_ts))


print('The accuracy_score of the model:',accuracy_score(y_pred,y_ts))


              precision    recall  f1-score   support

       False       0.37      0.59      0.46       868
        True       0.78      0.60      0.68      2132

    accuracy                           0.59      3000
   macro avg       0.58      0.59      0.57      3000
weighted avg       0.66      0.59      0.61      3000
 
Confusion Matrix : 
 [[ 515  353]
 [ 862 1270]] 

The f1_score of the model: 0.6764314247669774
The accuracy_score of the model: 0.595


##### Cross validarion on Train dataset XG Boost

In [63]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(random_search, X_tr, y_tr, cv=5, scoring='f1_macro')
scores.mean()

Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  19 out of  25 | elapsed:  2.6min remaining:   49.5s
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:  3.0min finished


Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  19 out of  25 | elapsed:  2.4min remaining:   45.3s
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:  3.3min finished


Fitting 5 folds for each of 5 candidates, totalling 25 fits


...

Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  19 out of  25 | elapsed:  3.2min remaining:  1.0min
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:  3.6min finished


Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  19 out of  25 | elapsed:  4.7min remaining:  1.5min
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:  5.2min finished


0.8812935332752637

<h1><center>Topic Modelling</center></h1>

In [92]:
import re
import nltk

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF

In [82]:
twd_sample.head()

,inbound,text
2070000,True,"@idea_cares Sneha, when i call some i hear you..."
2070001,False,@336024 We apologies for the trouble. We would...
2070002,True,"@idea_cares u irritating idiots, y the fuck pr..."
2070003,False,@307570 Sorry we gave you a chance of being up...
2070004,True,@idea_cares He told me - Sir don't repeat your...


- Checking the total number of observations and features

In [83]:
twd_sample.shape

(10000, 2)

In [85]:
twd_sample['text']=twd_sample['text'].apply(remove_noise)

- Text data after cleaning

In [87]:
twd_sample.head()

,inbound,text
2070000,True,idea care sneha call hear irritating unlimited...
2070001,False,apology trouble would like inform pca reverse ...
2070002,True,idea care u irritating idiot fuck precall anno...
2070003,False,sorry gave chance upset service weve noted con...
2070004,True,idea care told sir dont repeat issue followed ...


###### Considering only the Negative topics in the data set

In [88]:
twd_sample_negative=twd_sample[twd_sample['inbound']==False]
twd_sample_negative.head()

,inbound,text
2070001,False,apology trouble would like inform pca reverse ...
2070003,False,sorry gave chance upset service weve noted con...
2070008,False,troy sorry disappointing experience hope give ...
2070010,False,time feed need molly deliciousness awaits
2070012,False,sorry inconvenience checked offer applicable h...


###### Considering only the Positive topics in the data set

In [89]:
twd_sample_positive=twd_sample[twd_sample['inbound']==True]
twd_sample_positive.head()

,inbound,text
2070000,True,idea care sneha call hear irritating unlimited...
2070002,True,idea care u irritating idiot fuck precall anno...
2070004,True,idea care told sir dont repeat issue followed ...
2070005,True,idea care darshan tried seek help one supervis...
2070006,True,idea care darshan extremely serious issue firs...


In [90]:
def tokenize(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    
    filtered_tokens = []
    
    
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    
    return filtered_tokens

- Intial check of the function

In [93]:
tokenize(twd_sample_negative.iloc[0,1])

['apology',
 'trouble',
 'would',
 'like',
 'inform',
 'pca',
 'reverse',
 'pca',
 'playing',
 'someone',
 'try',
 'call',
 'regardssneha']

#### Tfidf Vectorizer with unigrams


In [94]:
term_idf_vectorizer      =     TfidfVectorizer(stop_words = 'english',max_features=2000, min_df=0.0005,\
                                      max_df=0.99,tokenizer = tokenize)
term_idf_matrix          =     term_idf_vectorizer.fit_transform(twd_sample_negative.text) 
term_idf_feature_names   =     term_idf_vectorizer.get_feature_names()


## Non Negative Matrix Factorization
- NNMF is an unsuperviser algorithm that performs dimensionality reduction and clustering
- We use this in conjuction with TF-IDF to model topics across documents


In [96]:
nmf_model=NMF(n_components=15,init='nndsvd',random_state=22)
nmf_model.fit(term_idf_matrix)

NMF(init='nndsvd', n_components=15, random_state=22)

In [97]:
for index, topic in enumerate(nmf_model.components_):
    print(f"The top 15 words for topic # {index}")
    print([term_idf_feature_names[i] for i in topic.argsort()[-15:]])
    print('\n')

The top 15 words for topic # 0
['r', 'info', 'use', 'f', 'information', 'woafwenp', 'check', 'j', 'h', 'wkjhdxwgrq', 'c', 'u', 'link', 'tco', 'http']


The top 15 words for topic # 1
['linked', 'associated', 'apologize', 'right', 'look', 'trouble', 'amp', 'connect', 'send', 'account', 'u', 'follow', 'dm', 'address', 'email']


The top 15 words for topic # 2
['device', 'happened', 'aw', 'try', 'sure', 'follow', 'resolved', 'additional', 'twitter', 'going', 'question', 'letting', 'u', 'let', 'know']


The top 15 words for topic # 3
['frustration', 'oh', 'way', 'really', 'order', 'tell', 'caused', 'trouble', 'inconvenience', 'delay', 'experience', 'store', 'im', 'hear', 'sorry']


The top 15 words for topic # 4
['shortly', 'right', 'contact', 'member', 'possible', 'support', 'patience', 'share', 'soon', 'appreciate', 'feedback', 'connect', 'touch', 'note', 'team']


The top 15 words for topic # 5
['ah', 'fine', 'ticket', 'speak', 'wish', 'andy', 'nice', 'colleague', 'start', 'longer', 'ha

#### Tfidf Vectorizer with bigrams anfd tri grams

In [98]:
topic_results=nmf_model.transform(term_idf_matrix)

In [99]:
term_idf_vectorizer     = TfidfVectorizer(max_df=0.99, max_features=2000,min_df=0.0005,\
                                          stop_words='english', use_idf=True, tokenizer=tokenize, ngram_range=(2,3))
term_idf_matrix         = term_idf_vectorizer.fit_transform(twd_sample_negative.text) 
term_idf_feature_names  = term_idf_vectorizer.get_feature_names()
term_idf_matrix.shape

nmf_model=NMF(n_components=15,init='nndsvd',random_state=22)
nmf_model.fit(term_idf_matrix)

(4572, 2000)

In [100]:
for index, topic in enumerate(nmf_model.components_):
    print(f"The top 15 words for topic # {index}")
    print([term_idf_feature_names[i] for i in topic.argsort()[-15:]])
    print('\n')

The top 15 words for topic # 0
['send dm', 'inconvenience caused', 'update available', 'hear send', 'inconvience caused', 'weve got', 'delivery date', 'issue device', 'http tco e', 'welcome let', 'connect u', 'u phone chat', 'let work', 'twitter english help', 'id love']


The top 15 words for topic # 1
['letting know', 'assistance thank', 'apple music', 'sound good', 'link provided', 'u know send', 'apologize trouble send', 'dm store', 'tco jojkltck', 'account info', 'u phone chat', 'help u', 'follow prompt', 'address account', 'glad able']


The top 15 words for topic # 2
['fb http tco', 'http tco ibiy', 'best help', 'u make', 'tracking number delivery', 'help u', 'sorry hear follow', 'address ac http', 'u note http', 'hi tony', 'season official', 'let know issue', 'u phone chat', 'let know', 'know issue persists']


The top 15 words for topic # 3
['hi dm u', 'order number', 'want make', 'send zip', 'persists http', 'u account email', 'critical response line', 'u know send', 'inapp s

In [101]:
twd_sample_negative['Topic']=topic_results.argmax(axis=1)

C:\Users\pc\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [102]:
twd_sample_negative.head()

,inbound,text,Topic
2070001,False,apology trouble would like inform pca reverse ...,10
2070003,False,sorry gave chance upset service weve noted con...,3
2070008,False,troy sorry disappointing experience hope give ...,3
2070010,False,time feed need molly deliciousness awaits,14
2070012,False,sorry inconvenience checked offer applicable h...,3


## People related to topic 4 are segregated 

In [103]:
twd_sample_negative[twd_sample_negative['Topic']==4]

,inbound,text,Topic
2070045,False,apology trouble request share complete address...,4
2070084,False,sorry problem service understand frustrating f...,4
2070122,False,oh dear sorry chris feedback logged passed mar...,4
2070150,False,hi richard escalated store maintenance team sm...,4
2070160,False,get food online team look smaira,4
2070199,False,account security team would best able assist c...,4
2070465,False,hey ross team looking currently please stay tu...,4
2070578,False,apologize trouble send u note http tco eyc uue...,4
2070580,False,sorry trouble send u note http tco juscquh tea...,4
2070582,False,take look send u note via http tco tv psvy sr ...,4
